### https://www.kaggle.com/janmejaybhoi/cotton-disease-dataset/activity

# Cotton Disease prediction

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [19]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential


In [4]:
from glob import glob

In [5]:
IMAGE_SIZE = [244,244]
train_path='train'
test_path='test'

In [6]:
train_path

'train'

In [7]:
resnet = ResNet50(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)

In [8]:
# don't train all layer
for layer in resnet.layers:
    layer.trainable = True

In [9]:
# output class
folders = glob('train\*')
folders

['train\\diseased cotton leaf',
 'train\\diseased cotton plant',
 'train\\fresh cotton leaf',
 'train\\fresh cotton plant']

In [10]:
# for all late number rows
x = Flatten()(resnet.output)

In [11]:
prediction = Dense(len(folders), activation='softmax')(x)

In [12]:
# create model 
model = Model(inputs=resnet.input, outputs=prediction)

In [13]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 244, 244, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 250, 250, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 122, 122, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 122, 122, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [14]:
# compile
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy']
             )

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range= 0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
# reading images
path1='train'
training_set = train_datagen.flow_from_directory(path1,
                                                target_size=(244,244),
                                                 batch_size=64,
                                                class_mode= 'categorical')

Found 1951 images belonging to 4 classes.


In [17]:
# reading images
path2='test'
test_set = test_datagen.flow_from_directory(path2,
                                            target_size=(244,244),
                                            batch_size=64,
                                            class_mode= 'categorical')

Found 106 images belonging to 4 classes.


In [18]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=2,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\Dell\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
31/31 [==============================] - 1490s 48s/step - loss: 4.3522 - accuracy: 0.6422 - val_loss: 263749936.0000 - val_accuracy: 0.2642
Epoch 2/2
31/31 [==============================] - 1326s 43s/step - loss: 1.6433 - accuracy: 0.8416 - val_loss: 61881.8594 - val_accuracy: 0.2642
